Google Analytics Customer Revenue - https://www.kaggle.com/c/ga-customer-revenue-prediction
1) Two Data sets, train and test
2) Entity embeddings of categorical variables
3) y = TransactionRevenue, for given x=fullVisitorId

Categorical Variables
['channelGrouping', 'device.browser', 'device.deviceCategory', 'geoNetwork.city', 'device.operatingSystem', 
                'trafficSource.medium', 'trafficSource.source',
                'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',  'geoNetwork.subContinent']

Continues Variables
 ['totals.visits', '_visitHour', '_day', '_month', '_weekday']

json columns ['device', 'geoNetwork', 'totals', 'trafficSource']

columns to drop   ['customDimensions','hits', 'visitId', 'totals.sessionQualityDim']

Submission - Root Mean Squared Error (RMSE)
For each fullVisitorId in the test set, you must predict the natural log of their total revenue in PredictedLogRevenue
fullVisitorId,PredictedLogRevenue

In [1]:
from os import path
from fastai.text import *

In [6]:
import pandas as pd # to manipulate data frames 
import numpy as np # to work with matrix
import json # to convert json in df
from pandas.io.json import json_normalize # to normalize the json fi

In [2]:
# json columns
columns = ['device', 'geoNetwork', 'totals', 'trafficSource']

In [3]:
PATH=Path('../data/kg-google/')

In [4]:
def json_read(filename):
    df = pd.read_csv(PATH/filename,
                     converters={column: json.loads for column in columns},
                     dtype={'fullVisitorId': 'str'},
#                      skiprows=lambda i : i> 0 and random.random() > p
                     nrows=20
                    )
    
    for column in columns:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f'{column}.{subcolumn}' for subcolumn in  column_as_df.columns]
        df = df.drop(column,axis=1).merge(column_as_df,right_index=True,left_index=True)
        
#     print(f'Loaded {os,path.basename(data_frame)}. Shape: {df.shape}')
    return df    


In [17]:
df_train = json_read('train_v2.csv')

In [8]:
df_train.size

1020

In [9]:
df_train

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,totals.timeOnSite,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508198450,1,1508198450,Firefox,...,NaN,1,NaN,not available in demo dataset,(not set),NaN,water bottle,organic,NaN,google
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1508176307,6,1508176307,Chrome,...,28,1,NaN,not available in demo dataset,(not set),NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508201613,1,1508201613,Chrome,...,38,1,NaN,not available in demo dataset,(not set),True,NaN,(none),NaN,(direct)
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,9075655783635761930,"[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,1508169851,1,1508169851,Chrome,...,1,1,NaN,not available in demo dataset,(not set),NaN,(not provided),organic,NaN,google
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,6960673291025684308,"[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,1508190552,1,1508190552,Chrome,...,52,1,NaN,not available in demo dataset,(not set),NaN,(not provided),organic,NaN,google
5,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,0166277907528479249,"[{'hitNumber': '1', 'time': '0', 'hour': '16',...",Not Socially Engaged,1508196701,1,1508196701,Chrome,...,12,1,NaN,not available in demo dataset,(not set),NaN,NaN,(none),/offer/2145,(direct)
6,Referral,"[{'index': '4', 'value': 'EMEA'}]",20171016,8349655975937271469,"[{'hitNumber': '1', 'time': '0', 'hour': '4', ...",Not Socially Engaged,1508152478,1,1508152478,Chrome,...,9,1,NaN,not available in demo dataset,(not set),NaN,NaN,referral,/a/google.com/nest-vision/dropcam-field-tester...,sites.google.com
7,Organic Search,[],20171016,1332629902468998662,"[{'hitNumber': '1', 'time': '0', 'hour': '19',...",Not Socially Engaged,1508206208,1,1508206208,Chrome,...,15,1,NaN,not available in demo dataset,(not set),NaN,(not provided),organic,NaN,google
8,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,632878546807742341,"[{'hitNumber': '1', 'time': '0', 'hour': '19',...",Not Socially Engaged,1508207516,1,1508207516,Chrome,...,34,1,NaN,not available in demo dataset,(not set),NaN,(not provided),organic,NaN,google
9,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,1259490915281096752,"[{'hitNumber': '1', 'time': '0', 'hour': '7', ...",Not Socially Engaged,1508165159,2,1508165159,Safari,...,32,1,NaN,not available in demo dataset,(not set),True,(not provided),organic,NaN,google


In [10]:
df_train.columns

Index(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId', 'hits',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.browser', 'device.browserSize', 'device.browserVersion',
       'device.deviceCategory', 'device.flashVersion', 'device.isMobile',
       'device.language', 'device.mobileDeviceBranding',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.operatingSystem', 'device.operatingSystemVersion',
       'device.screenColors', 'device.screenResolution', 'geoNetwork.city',
       'geoNetwork.cityId', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.networkLocation',
       'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.ses

In [18]:
#columns to drop
to_drop = ['customDimensions','hits', 'visitId', 'totals.sessionQualityDim', 'visitNumber','device.browserVersion','device.browserSize',
          'device.language', 'device.mobileDeviceBranding','device.mobileDeviceInfo', 'device.mobileDeviceMarketingName', 'device.mobileDeviceModel', 'device.mobileInputSelector',
          'device.operatingSystemVersion','device.screenColors', 'device.screenResolution', 'geoNetwork.latitude', 'geoNetwork.longitude',
          'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.networkLocation','totals.sessionQualityDim', 'trafficSource.adwordsClickInfo.criteriaParameters',
           'trafficSource.adContent','trafficSource.referralPath', 'trafficSource.source', 'trafficSource.medium'
          ]

In [19]:
df_train.drop(to_drop, axis=1, inplace=True)

In [20]:
df_train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'socialEngagementType',
       'visitStartTime', 'device.browser', 'device.deviceCategory',
       'device.flashVersion', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.cityId', 'geoNetwork.continent',
       'geoNetwork.country', 'geoNetwork.region', 'geoNetwork.subContinent',
       'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.timeOnSite', 'totals.visits', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword'],
      dtype='object')